In [ ]:
# Tests for methods that process horizon surface (filtering, creating carcass, holes, etc.)
import os
import sys
import warnings
import numpy as np
from scipy.ndimage.morphology import binary_dilation

warnings.filterwarnings('ignore')

sys.path.append('../../../seismiqb')

from seismiqb import Field, Horizon, plot_image

In [ ]:
""" You can manage cube and horizon for the test:

CUBE_PATH : str
    Path to an existed seismic cube.
HORIZON_PATH : str
    Path to an existed seismic horizon.
"""
# Tests parameters
OUTPUT_DIR = './horizon_test_files'

CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')
HORIZON_PATH = os.path.join(OUTPUT_DIR, 'test_horizon')

SEED = 42

# Visualization parameters
SCALE = 1
SHOW_FIGURES = True

In [ ]:
%%time
field = Field(CUBE_PATH)
horizon = Horizon(HORIZON_PATH, field=field)

horizon.show(show=SHOW_FIGURES)

# Filtering

In [ ]:
%%time
# from_subset
# Cut horizon by i_line and check its filtered and non-filtered parts
# Create filtering matrix
i_line_cut = horizon.shape[0] // 3

filtering_matrix = np.zeros_like(horizon.full_matrix)
filtering_matrix[i_line_cut:] = 1

# Horizon filtered by matrix
filtered_horizon = Horizon(storage=horizon.full_matrix, field=field, name='filtered')
filtered_horizon = filtered_horizon.from_subset(filtering_matrix)

filtered_horizon.show(show=SHOW_FIGURES)

not_empty_traces_error = "`from_subset` filtering test failed: filtered traces are not empty"
assert np.all(filtered_horizon.full_matrix[:i_line_cut] == filtered_horizon.FILL_VALUE), not_empty_traces_error

empty_traces_error = "`from_subset` filtering test failed: non-filtered traces was changed by filtering"
assert np.array_equal(filtered_horizon.full_matrix[i_line_cut:],
                      horizon.full_matrix[i_line_cut:]), empty_traces_error

In [ ]:
%%time
# filter
filtered_horizon = Horizon(horizon.full_matrix, field=field, name='filtered')
filtered_horizon.filter()

filtered_horizon.show(show=SHOW_FIGURES)

# Check that zero_traces filled by FILL_VALUE
zero_traces = filtered_horizon.full_matrix[field.zero_traces == 1]

not_empty_traces_error = "`filter` test failed: filtered traces are not empty"
assert np.all(zero_traces == filtered_horizon.FILL_VALUE), not_empty_traces_error

# Check that other traces didn't change
nonzero_traces = filtered_horizon.full_matrix[field.zero_traces == 0]
original_nonzero_traces = horizon.full_matrix[field.zero_traces == 0]

empty_traces_error = "`filter` test failed: non-filtered traces was changed by filtering"
assert np.array_equal(nonzero_traces, original_nonzero_traces), empty_traces_error

In [ ]:
%%time
# filter_matrix
# Cut horizon by i_line and check its filtered and non-filtered parts
i_line_cut = horizon.shape[0] // 3

filtered_horizon = Horizon(horizon.full_matrix, field=field, name='filtered')

filtering_matrix = np.zeros_like(horizon.full_matrix)
filtering_matrix[:i_line_cut] = 1

filtered_horizon.filter_matrix(filtering_matrix)

filtered_horizon.show(show=SHOW_FIGURES)

not_empty_traces_error = "`filter_matrix` filtering test failed: filtered traces are not empty"
assert np.all(filtered_horizon.full_matrix[:i_line_cut] == filtered_horizon.FILL_VALUE), not_empty_traces_error

empty_traces_error = "`filter_matrix` filtering test failed: non-filtered traces was changed by filtering"
assert np.array_equal(filtered_horizon.full_matrix[i_line_cut:],
                      horizon.full_matrix[i_line_cut:]), empty_traces_error

In [ ]:
%%time
# filter_points
# Cut horizon by i_line and check its filtered and non-filtered parts
i_line_cut = horizon.shape[0] // 3

filtered_horizon = Horizon(horizon.full_matrix, field=field, name='filtered')

filtering_matrix = np.zeros_like(horizon.full_matrix)
filtering_matrix[:i_line_cut] = 1

filtered_horizon.filter_points(filtering_matrix)

filtered_horizon.show(show=SHOW_FIGURES)

not_empty_traces_error = "`filter_matrix` filtering test failed: filtered traces are not empty"
assert np.all(filtered_horizon.full_matrix[:i_line_cut] == filtered_horizon.FILL_VALUE), not_empty_traces_error

empty_traces_error = "`filter_matrix` filtering test failed: non-filtered traces was changed by filtering"
assert np.array_equal(filtered_horizon.full_matrix[i_line_cut:],
                      horizon.full_matrix[i_line_cut:]), empty_traces_error

In [ ]:
%%time
# Filter spikes by maps
# Create constant horizon with a spike
spike_point = (horizon.shape[0] // 2, horizon.shape[1] // 2)

horizon_matrix = np.ones_like(horizon.full_matrix) * 1000
horizon_matrix[spike_point] += 5

spiked_horizon = Horizon(horizon_matrix, field=field, name='constant_with_spike')
spiked_horizon.filter()

# Get mask of areas with spike cut (only for dilation=1)
dilation = 1
spikes_mask = np.zeros(shape=spiked_horizon.full_matrix.shape, dtype=bool)
spikes_mask[spike_point] = 1            
spikes_mask = binary_dilation(spikes_mask, iterations=dilation)

spikes_mask_gradient = spikes_mask.copy() # it has i_line and x_line shifts
spikes_mask_gradient[1:] += spikes_mask[:-1]
spikes_mask_gradient[:, 1:] += spikes_mask[:, :-1]

spikes = {
    'median': spikes_mask,
    'gradient': spikes_mask_gradient
}

# Check despiking
for mode, spikes_mask in spikes.items():
    despiked_horizon = Horizon(spiked_horizon.full_matrix, field=field, name='despiked_constant')

    despiked_horizon.filter_spikes(mode=mode, dilation=1)

    not_empty_traces_error = f"Despike with mode '{mode}' test failed: filtered traces with spike are not empty"
    assert np.all(despiked_horizon.full_matrix[spikes_mask] == despiked_horizon.FILL_VALUE), not_empty_traces_error

    empty_traces_error = f"Despike with mode '{mode}' test failed: despike changes traces without spikes"
    assert np.array_equal(despiked_horizon.full_matrix[~spikes_mask], spiked_horizon.full_matrix[~spikes_mask]), empty_traces_error

    print(f"Despiking mode '{mode}' was successfully tested")

In [ ]:
%%time
# filter_disconnected_regions
# Split horizon in two parts by i_line_cut
i_line_cut = horizon.shape[0]//3
filtering_matrix = np.zeros_like(horizon.full_matrix)
filtering_matrix[i_line_cut, :] = 1

splitted_horizon = Horizon(horizon.full_matrix, field=field, name='splitted')
splitted_horizon.filter(filtering_matrix)

splitted_horizon.filter_disconnected_regions()

not_empty_traces_error = f"`filter_disconnected_regions` test failed: disconnected region wasn't filtered"
assert np.all(splitted_horizon.full_matrix[:i_line_cut+1]==splitted_horizon.FILL_VALUE), not_empty_traces_error

empty_traces_error = f"`filter_disconnected_regions` test failed: wrong traces were filtered"
assert np.array_equal(splitted_horizon.full_matrix[i_line_cut+1:],  horizon.full_matrix[i_line_cut+1:]), empty_traces_error

# Specific manipulations

In [ ]:
horizon = Horizon(horizon.full_matrix, field=field, name='filtered')
horizon.filter()

frequency = 100

def calculate_grid_coverage(horizon, frequencies=100, width=1, **kwargs):
    """ Approximate calculation of coverage of regular grid.

    horizon : :class:`Horizon`
        Seismic horizon instance.
    frequencies : int or sequence of two integers
        (iline, xline) grid frequencies.
        If int, then ilines and xlines frequencies are equal.
    width : int or sequence of two integers
        (iline, xline) grid width.
        If int, then ilines and xlines grid widths are equal.
    """
    if isinstance(frequencies, int):
        frequencies = (frequencies, frequencies)

    i_min = kwargs.get('i_min', horizon.i_min)
    x_min = kwargs.get('x_min', horizon.x_min)

    amount_of_ilines = (horizon.i_length - i_min) // frequencies[0]
    amount_of_xlines = (horizon.x_length - x_min) // frequencies[1]

    # On borders we have a line with half of grid width
    if (horizon.i_length - i_min) % frequencies[0] > 0:
        amount_of_ilines += 0.5
    else:
        amount_of_ilines -= 0.5

    if (horizon.x_length - x_min) % frequencies[1] > 0:
        amount_of_xlines += 0.5
    else:
        amount_of_xlines -= 0.5

    # Count amount of traces for grid with width = 1
    intersection_traces = amount_of_ilines * amount_of_xlines
    grid_traces = amount_of_ilines * horizon.x_length + amount_of_xlines * horizon.i_length

    # Apply width
    intersection_traces = width * width * intersection_traces
    grid_traces = width * grid_traces
    grid_traces -= intersection_traces

    # Calculate approximate amount of traces in grid in horizon area
    coverage = grid_traces / horizon.size
    return coverage

## Carcass

In [ ]:
%%time
carcass = horizon.make_carcass(margin=0, frequencies=frequency, regular=True, apply_smoothing=True)

carcass.show(show=SHOW_FIGURES, load_kwargs={'enlarge': True})

approximate_coverage = calculate_grid_coverage(horizon=horizon, frequencies=frequency, width=3, i_min=0, x_min=0)

assert np.isclose(carcass.coverage, approximate_coverage, atol=2e-3), "`make_carcass` test failed: resulted coverage is not expected"

## thin_out

In [ ]:
%%time
thined_horizon = Horizon(horizon.full_matrix, field=field, name='thined')
thined_horizon.thin_out(factor=(frequency, frequency), threshold=np.min(horizon.shape)//10)

thined_horizon.show(show=SHOW_FIGURES, load_kwargs={'enlarge': True})

approximate_coverage = calculate_grid_coverage(horizon=horizon, frequencies=frequency, width=1,
                                               i_min=horizon.i_min, x_min=horizon.x_min)

assert np.isclose(thined_horizon.coverage, approximate_coverage, atol=2e-3), "`thin_out` test failed: resulted coverage is not expected"

horizon# Interpolate

In [ ]:
%%time
interpolated_horizon = Horizon(thined_horizon.full_matrix, field=field, name='interpolated')
interpolated_horizon.interpolate()

interpolated_horizon.show(show=SHOW_FIGURES, load_kwargs={'enlarge': True})

approximate_coverage = calculate_grid_coverage(horizon=horizon, frequencies=frequency, width=3,
                                               i_min=horizon.i_min, x_min=horizon.x_min)

assert np.isclose(interpolated_horizon.coverage, approximate_coverage, atol=2e-3), "`interpolate` test failed: resulted coverage is not expected"

## Holes

In [ ]:
%%time
filtering_matrix = horizon.make_random_holes_matrix(seed=SEED)
filtering_matrix = horizon.matrix_put_on_full(filtering_matrix)

horizon_with_holes = Horizon(horizon.full_matrix, field=field, name='holed')
horizon_with_holes.filter(filtering_matrix)

if SHOW_FIGURES:
    plot_image(filtering_matrix, cmap='viridis', scale=SCALE, title='Holes matrix')
    horizon_with_holes.show(scale=SCALE)

assert (horizon_with_holes.coverage < horizon.coverage), "`make_random_holes_matrix` test failed: no traces was filtered"